### AB Model from spec

Package installation

In [1]:
# !pip3 install seaborn
# !pip3 install python-statemachine
# !pip3 install mesa
# !pip3 install transitions
# !pip3 install scipy
# !pip3 install cufflinks
# !pip3 install graphviz pygraphviz
# !pip3 install graphviz
# !pip3 install transitions[diagrams]
# !pip3 uninstall mesa

Package importation

In [2]:
# imports
import os
import seaborn as sns
from random import choice
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import mesa
from mesa import Agent, Model
from mesa.time import RandomActivation, RandomActivationByType, SimultaneousActivation
from mesa.datacollection import DataCollector
from matplotlib import pyplot as plt, patches
import scipy.stats as ss
import cufflinks as cf
cf.go_offline()
from plotly.offline import iplot
from transitions import Machine
import random
from transitions.extensions import GraphMachine
import graphviz
import timeit

import logging
# os.environ["PATH"] += os.pathsep + '/Users/ia329/homebrew/bin' # for graphviz

Model component importation

In [4]:
from EV.agent import EV, ChargeStation
import EV.model as model
from EV.statemachine import EVSM, LSM
from EV.model import get_evs_charge, get_evs_charge_level, get_evs_active, get_evs_queue, get_evs_travel, get_evs_not_idle, get_active_chargestations, get_eod_evs_socs, get_evs_destinations, get_ev_distance_covered
# old:  get_evs_charging,

State diagram

### Model

Model parameters

In [5]:
ticks =  48 # 3600 ticks = 3600 seconds = 1 hour
no_evs = 20
# no of css must be fixed
no_css = 5

Run Model

In [6]:
model_run = model.EVModel(ticks=ticks, no_evs=no_evs, no_css=no_css)
for i in range(ticks):
    model_run.step()


EV info: ID: 0, destination name: City A, journey type: InterUrban, max_battery: 46, speed: 20, State: Idle.
EV info (Cont'd): Start time: 6, distance goal: 120, energy consumption rate: 0.5, charge prop 0.4, location: City_D.

EV info: ID: 1, destination name: City C, journey type: InterUrban, max_battery: 60, speed: 20, State: Idle.
EV info (Cont'd): Start time: 7, distance goal: 150, energy consumption rate: 0.5, charge prop 0.4, location: City_D.

EV info: ID: 2, destination name: friend_2, journey type: Urban, max_battery: 68, speed: 10, State: Idle.
EV info (Cont'd): Start time: 7, distance goal: 90, energy consumption rate: 0.2, charge prop 0.4, location: City_D.

EV info: ID: 3, destination name: work, journey type: Urban, max_battery: 42, speed: 10, State: Idle.
EV info (Cont'd): Start time: 14, distance goal: 50, energy consumption rate: 0.2, charge prop 0.4, location: City_D.

EV info: ID: 4, destination name: work, journey type: Urban, max_battery: 66, speed: 10, State: Id

In [7]:
run_stats = model_run.datacollector.get_model_vars_dataframe()
print(run_stats)

    EVs Charging  EVs Activated  EVs Travelling  EVs Queued  EVs Dead  \
0              0             20               0           0         0   
1              0             20               0           0         0   
2              0             20               0           0         0   
3              0             20               0           0         0   
4              0             20               0           0         0   
5              0             20               0           0         0   
6              0             20               5           0         0   
7              0             20               8           0         0   
8              0             20               8           0         0   
9              2             20               6           1         0   
10             2             20               6           1         1   
11             3             20               4           1         2   
12             3             20               7    

In [ ]:
a = 26 % 24
print(a) 

Export results to CSV file

In [ ]:
model_run.datacollector.get_model_vars_dataframe().to_csv('02_03_20EV_agent_model_output.csv')

In [ ]:
# def unpack_datacollector(dc):
#     """
#     Unpacks the data from a DataCollector instance in Mesa.
    
#     Args:
#         dc (DataCollector): A DataCollector instance in Mesa.
    
#     Returns:
#         A dictionary with the keys as the variable names and the values as the lists of data for each variable.
#     """
#     data = {}
#     for varname, var in dc.model_vars.items():
#         data[varname] = var.get_data()
#     return data

### Data import and preprocessing

Helper functions. May eventually move to external module

In [8]:
def split_column_values(df, col_name):
    """
    Splits the values in a column by commas and creates a new column for each value.

    Args:
        df (pandas.DataFrame): A pandas DataFrame.
        col_name (str): The name of the column to split.

    Returns:
        A pandas DataFrame with additional columns for each value in the input column.
    """
    # Get the unique values in the column
    unique_values = set(df[col_name].str.cat(sep=',').split(','))

    # Create a new column for each unique value
    for value in unique_values:
        df[value] = df[col_name].str.contains(value).astype(int)

    # Drop the original column
    df.drop(col_name, axis=1, inplace=True)

    return df


def unpack_and_join(df, column_name):
    """
    Unpacks the values in a column by commas and creates a new column for each value. Removes square brackets from the values.

    Args:
        df (pandas.DataFrame): A pandas DataFrame.
        column_name (str): The name of the column to unpack.

    Returns:
        A pandas DataFrame with additional columns for each value in the input column.
        
    """
    # Get the column values as a list of strings
    column_values = df[column_name].tolist()

    # Strip the square brackets from the strings
    column_values = [s.strip("[]") for s in column_values]

    # Split the strings on commas and create a list of lists
    split_values = [s.split(",") for s in column_values]

    # Get the number of columns needed
    num_cols = max([len(row) for row in split_values])

    # Create the new columns in the output dataframe
    column_names = [column_name+"_unpacked_"+str(i) for i in range(num_cols)]
    new_df = pd.DataFrame(columns=column_names)

    # Loop over the original column values and add the unpacked values to the new dataframe
    for vals in split_values:
        row_data = {}
        for i in range(num_cols):
            if i < len(vals):
                row_data[column_name+"_unpacked_"+str(i)] = vals[i].strip()
            else:
                row_data[column_name+"_unpacked_"+str(i)] = ""
        new_df = new_df.append(row_data, ignore_index=True)

    # Merge the original dataframe with the new unpacked dataframe
    merged_df = pd.concat([df, new_df], axis=1)

    return merged_df


In [9]:
data = pd.read_csv('01_03_20EV_agent_model_output.csv')

In [10]:
# data.info()
data.head()

,Unnamed: 0,EVs Charging,EVs Activated,EVs Travelling,EVs Queued,EVs Charge Level,EVs Currently charging,EVs Not Idle,EOD Battery SOC,EVs Destinations,EVs at Charging Station - S,Length of Queue 1 at Charging Stations,Length of Queue 2 at Charging Stations,EVs at Charging Stations
0,0,0,20,0,0,"[63, 41, 57, 49, 62, 43, 52, 49, 68, 68, 61, 5...",0,0,"[[53.0, 43.0], [18.5, 6.0], [17.0, 7.0], [26.5...","['work', 'City C', 'City B', 'City A', 'friend...",0,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",NaN
1,1,0,20,0,0,"[63, 41, 57, 49, 62, 43, 52, 49, 68, 68, 61, 5...",0,0,"[[53.0, 43.0], [18.5, 6.0], [17.0, 7.0], [26.5...","['work', 'City C', 'City B', 'City A', 'friend...",0,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",NaN
2,2,0,20,0,0,"[63, 41, 57, 49, 62, 43, 52, 49, 68, 68, 61, 5...",0,0,"[[53.0, 43.0], [18.5, 6.0], [17.0, 7.0], [26.5...","['work', 'City C', 'City B', 'City A', 'friend...",0,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",NaN
3,3,0,20,0,0,"[63, 41, 57, 49, 62, 43, 52, 49, 68, 68, 61, 5...",0,0,"[[53.0, 43.0], [18.5, 6.0], [17.0, 7.0], [26.5...","['work', 'City C', 'City B', 'City A', 'friend...",0,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",NaN
4,4,0,20,0,0,"[63, 41, 57, 49, 62, 43, 52, 49, 68, 68, 61, 5...",0,0,"[[53.0, 43.0], [18.5, 6.0], [17.0, 7.0], [26.5...","['work', 'City C', 'City B', 'City A', 'friend...",0,"[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]",NaN


In [11]:
a = unpack_and_join(data, 'EVs Charge Level')
print(a)

    Unnamed: 0  EVs Charging  EVs Activated  EVs Travelling  EVs Queued  \
0            0             0             20               0           0   
1            1             0             20               0           0   
2            2             0             20               0           0   
3            3             0             20               0           0   
4            4             0             20               0           0   
5            5             0             20               0           0   
6            6             0             20               3           0   
7            7             0             20               5           0   
8            8             0             20               8           0   
9            9             0             20               9           0   
10          10             1             20               9           0   
11          11             1             20              11           1   
12          12           

In [12]:
# EV charge level per EV per timestep - 20 EVs unpacked
newdf = a[['EVs Charge Level', 'EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4', 'EVs Charge Level_unpacked_5', 'EVs Charge Level_unpacked_6', 'EVs Charge Level_unpacked_7', 'EVs Charge Level_unpacked_8', 'EVs Charge Level_unpacked_9', 'EVs Charge Level_unpacked_10', 'EVs Charge Level_unpacked_11', 'EVs Charge Level_unpacked_12', 'EVs Charge Level_unpacked_13', 'EVs Charge Level_unpacked_14', 'EVs Charge Level_unpacked_15', 'EVs Charge Level_unpacked_16', 'EVs Charge Level_unpacked_17', 'EVs Charge Level_unpacked_18', 'EVs Charge Level_unpacked_19']]
newdf.head()

,EVs Charge Level,EVs Charge Level_unpacked_0,EVs Charge Level_unpacked_1,EVs Charge Level_unpacked_2,EVs Charge Level_unpacked_3,EVs Charge Level_unpacked_4,EVs Charge Level_unpacked_5,EVs Charge Level_unpacked_6,EVs Charge Level_unpacked_7,EVs Charge Level_unpacked_8,...,EVs Charge Level_unpacked_10,EVs Charge Level_unpacked_11,EVs Charge Level_unpacked_12,EVs Charge Level_unpacked_13,EVs Charge Level_unpacked_14,EVs Charge Level_unpacked_15,EVs Charge Level_unpacked_16,EVs Charge Level_unpacked_17,EVs Charge Level_unpacked_18,EVs Charge Level_unpacked_19
0,"[63, 41, 57, 49, 62, 43, 52, 49, 68, 68, 61, 5...",63,41,57,49,62,43,52,49,68,...,61,51,59,69,55,51,62,58,55,48
1,"[63, 41, 57, 49, 62, 43, 52, 49, 68, 68, 61, 5...",63,41,57,49,62,43,52,49,68,...,61,51,59,69,55,51,62,58,55,48
2,"[63, 41, 57, 49, 62, 43, 52, 49, 68, 68, 61, 5...",63,41,57,49,62,43,52,49,68,...,61,51,59,69,55,51,62,58,55,48
3,"[63, 41, 57, 49, 62, 43, 52, 49, 68, 68, 61, 5...",63,41,57,49,62,43,52,49,68,...,61,51,59,69,55,51,62,58,55,48
4,"[63, 41, 57, 49, 62, 43, 52, 49, 68, 68, 61, 5...",63,41,57,49,62,43,52,49,68,...,61,51,59,69,55,51,62,58,55,48


### Batching

In [ ]:
from mesa.batchrunner import BatchRunner

EVcounts = (100,500,1000)
cpcounts = (1,2)
tickcounts = (24,48)
model_reporters={'EVs Charged': get_evs_charged,
                'EVs Activated': get_evs_active,
                'EVs Travelling': get_evs_travel,
                'EVs Charge Level': get_evs_charge_level,
                'EVs Currently charging': get_evs_charging,
                'EVs Not Idle': get_evs_not_idle,
                'EOD Battery SOC': get_eod_evs_socs,
                'EVs Destinations': get_evs_destinations,
                }
# parameters = {"no_evs": range(1000,20000,3000), "no_cps": 1}
parameters = {"no_evs": EVcounts, "no_cps": cpcounts, "ticks": tickcounts}
batch_run = BatchRunner(model.EVModel, parameters, max_steps=24, iterations=1, model_reporters= model_reporters) #iterations=1
batch_run.run_all()

In [ ]:
# batch_df = batch_run.get_model_vars_dataframe()

In [ ]:
# print(batch_df)

### Visualisations

### Scrapbook